<a href="https://colab.research.google.com/github/DrakeShadowRaven/desihigh/blob/main/n'ish_body.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/desihigh')

In [4]:
import time
import astropy
import itertools
import matplotlib

import numpy             as np
import pylab             as pl
import matplotlib.pyplot as plt
import astropy.units     as u

from   astropy.cosmology import FlatLambdaCDM
from   IPython.display   import YouTubeVideo
from   tools.flops       import flops

In [5]:
from IPython.display import clear_output
from time import sleep

In [6]:
%matplotlib inline

plt.style.use('dark_background')

In [7]:
def init_dof(npt=1):
  #  Create a set of particles at random positions in a box, which will soon predict the distribution of dark matter 
  #  as we see above.
  xs     = np.random.uniform(0., 1., npt)
  ys     = np.random.uniform(0., 1., npt)
  zs     = np.random.uniform(0., 1., npt)

  pos    = np.vstack((xs,ys,zs)).T
  vel    = np.zeros_like(pos)

  return  pos, vel

In [61]:
# Reinitialise particles.
npt=2
pos, vel = init_dof(npt=npt)
mass = np.random.uniform(0., 1., npt)
msize = mass*20
print, mass

# force the last mass very small


print,mass
# pos of partile 1

i = 0
pos[i][0]=0.5
pos[i][1]=0.5
pos[i][2]=0.5
pos[0]=0

i = 1 
pos[i][0]=0.7
pos[i][1]=0.5
pos[i][2]=1.5e12

r = 1.5e12
mass_1 = 1.5e30
mass_2 = 6e30
v_1 = 13200
v_2 = -3300
print(r)
print(mass_1)
print(mass_2)
print(v_1)
print(v_2)

G = 6.67e-11

# vel of partile 1

i = 0
vel[i][0]=0.1
vel[i][1]=0.05
vel[i][2]=0.1
vel[0]=13200

i = 1 
vel[i][0]=0.05
vel[i][1]=0.1
vel[i][2]=0.2
vel[1]=-3300

mass[0]=6e30
mass[1]=1.5e30
print(vel)

p_1 = mass_1*v_1
p_2 = mass_2*v_2
print(p_1)
print(p_2)

momentum_1 = mass[0]*vel[0]
momentum_2 = mass[1]*vel[1]
print(momentum_1)
print(momentum_2)

dR = mass[0]*vel[0]+mass[1]*vel[1]
print(dR)
R = dR/(mass[0]*mass[1])
print(R)

E = .5*mass[0]*vel[0]**+.5*mass[1]*vel[1]**-(G*mass[0]*mass[1])/(r)
dE = .5*mass[0]*v_1**+.5*mass[1]*v_2**-(G*mass[0]*mass[1])/r
print(E)
print(dE)

L = mass[0]*vel[0]*1.2e12+mass[1]*vel[1]*.3e12
print(L)

1500000000000.0
1.5e+30
6e+30
13200
-3300
[[13200. 13200. 13200.]
 [-3300. -3300. -3300.]]
1.9799999999999997e+34
-1.9799999999999997e+34
[7.92e+34 7.92e+34 7.92e+34]
[-4.95e+33 -4.95e+33 -4.95e+33]
[7.425e+34 7.425e+34 7.425e+34]
[8.25e-27 8.25e-27 8.25e-27]
[0. 0. 0.]
0.0
[9.3555e+46 9.3555e+46 9.3555e+46]


In [22]:
def init_dof(npt=1):
  #  Create a set of particles at random positions in a box, which will soon predict the distribution of dark matter 
  #  as we see above.
  xs     = np.random.uniform(0., 1., npt)
  ys     = np.random.uniform(0., 1., npt)
  zs     = np.random.uniform(0., 1., npt)

  pos    = np.vstack((xs,ys,zs)).T
  vel    = np.zeros_like(pos)

  return  pos, vel

In [27]:

# Number and mass of particles
npt        = 3
#mass       = 0.25

# Whether to draw arrows for the acceleration and velocity
draw_acc   = False
#draw_vel   = False
draw_vel = True

# A small drag term to simulate the real drag dark matter particles experience due to the expanding universe
drag = 1e-2
dt         = 5e-4
nsteps     = 500
timesteps  = np.linspace(0, (nsteps)*dt, nsteps, endpoint=False)
def make_legend_arrow(legend, orig_handle,
          xdescent, ydescent,
          width, height, fontsize):
    p = matplotlib.patches.FancyArrow(0, 0.5*height, width, 0, length_includes_head=True, head_width=0.75*height)
    return p

for index_in_timestep, time in enumerate(timesteps):
    clear_output(wait=True)
    if index_in_timestep%10 == 1:
        fig, ax = plt.subplots(1,1, figsize=(5,5), dpi=150)
        ax.tick_params(labelbottom=False, labelleft=False, left=False, bottom=False)
        ax.clear()
        ax.set_title(f'N-body simulation with $N={npt}$ particles')
        step_label = ax.text(0.03, .97, f'Step {index_in_timestep}',
            transform=ax.transAxes, verticalalignment='top', c='k',
            bbox=dict(color='w', alpha=0.8))
        
    dvel                        = np.zeros_like(vel)
    dpos                        = np.zeros_like(pos)
    acc                         = np.zeros_like(pos)
    for index_in_particle in range(npt):
        acc[index_in_particle]  = g_at_pos(pos[index_in_particle], pos, mass, epsilon=0.1)
        # Update velocities.
        dvel[index_in_particle] = dt * acc[index_in_particle]
        # Update positions.
        dpos[index_in_particle] = dt * vel[index_in_particle]
    vel += dvel - drag*vel
    pos += dpos
    # Our simulation has periodic boundaries, if you go off one side you come back on the other!
    pos = pos % 1
    ax.scatter(pos[:,0], pos[:,1], s=msize, color='darkorange', edgecolor='w')
    # Draw arrows representing the velocity and acceleration vectors, if requested
    # The code here is a little verbose to get nice-looking arrows in the legend
    arrows = []
    if draw_vel:
        ax.quiver(pos[:,0], pos[:,1], vel[:,0], vel[:,1], color='w', zorder=0)
        arrows += [matplotlib.patches.FancyArrow(0,0, 0.5, 0.6, label='Velocity', color='w')]
    if draw_acc:
        ax.quiver(pos[:,0], pos[:,1], acc[:,0], acc[:,1], color='darkorange', zorder=0)
        arrows += [matplotlib.patches.FancyArrow(0,0, 0.5, 0.6, label='Accel', color='darkorange')]
    if draw_vel or draw_acc:
        ax.legend(handles=arrows, handler_map={matplotlib.patches.FancyArrow:matplotlib.legend_handler.HandlerPatch(patch_func=make_legend_arrow)},
                 facecolor='k', edgecolor='white', framealpha=0.8,
                 loc='lower right')
#    fig.canvas.draw()
    if index_in_timestep%10 == 1:
      ax.set_xlim(0.5, 0.7)
      ax.set_ylim(0.4, 0.6)
      plt.show(fig)
      sleep(0.05)

NameError: ignored